### Importing all the necessary libraries

In [1]:
import os
import glob
import cv2
import json
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import earthpy.plot as ep
import earthpy.spatial as es
from progressbar import ProgressBar
from matplotlib import pyplot as plt
%matplotlib inline
from mrc_insar_common.data import data_reader

Import function from local file

In [2]:
from dataset import readRas, data_path_split, patch_images, MyDataset, transform_data, Augment
from config import get_config

Get the hyperparameter dictionary from `config` file using `get_config()` function and creating directory for saving the figures.

In [3]:
config = get_config()
pathlib.Path(config['visualization_dir']).mkdir(parents=True, exist_ok=True)

Create csv and json file if not exits

In [ ]:
if not (os.path.exists(config['train_dir'])):
    data_path_split(config)
# creating jason files for train, test and validation
if not (os.path.exists(config["p_train_dir"])) and config['patchify']:
    print("Saving patchify indices for train and test.....")

    train_data = pd.read_csv(config['train_dir'])
    valid_data = pd.read_csv(config['valid_dir'])
    test_data = pd.read_csv(config['test_dir'])

    config['patch_class_balance'] = True
    patch_images(train_data, config, "train_patch_phr_cb_")
    patch_images(valid_data, config, "valid_patch_phr_cb_")
    patch_images(test_data, config, "test_patch_phr_cb_")

    config['patch_class_balance'] = False
    patch_images(train_data, config, "train_patch_phr_")
    patch_images(valid_data, config, "valid_patch_phr_")
    patch_images(test_data, config, "test_patch_phr_")


### Helper Functions

#### For ploting different type of images.

plot():

- display_list (dict) - a dictionary where keys are the title of the image and values are images.
- id (int) - image id.
- directory (str) - path to save the plot.

In [4]:
def plot(display_list, id, directory, show=True):
    plt.figure(figsize=(20, 16))
    title = list(display_list.keys())
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)

        # plot dem channel using earthpy
        if title[i]=="dem":
            ax = plt.gca()
            hillshade = es.hillshade(display_list[title[i]], azimuth=180)
            ep.plot_bands(
                display_list[title[i]],
                cbar=False,
                cmap="terrain",
                title=title[i],
                ax=ax
            )
            ax.imshow(hillshade, cmap="Greys", alpha=0.5)
        # gray image plot rslc\n",
        elif title[i]=="rslc1" or title[i]=="rslc2":
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]])**0.25, cmap="gray", interpolation=None)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
        elif title[i]=="rslc1_label" or title[i]=="rslc2_label":
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]])**0.3, cmap="gray", interpolation=None)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
        # ifr plot
        elif title[i]=="ifg":
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]]), cmap='YlGnBu', interpolation=None)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
        # plot labels
        else:
            plt.title(title[i])
            c2 = plt.imshow((display_list[title[i]]), vmin=0, vmax=1)
            ax = plt.gca()
            fig = plt.gcf()
            fig.colorbar(c2, ax=ax, fraction=0.046)
            plt.axis('off')
    
    prediction_name = "img_id_{}.png".format(id) # create file name to save
    plt.savefig(os.path.join(directory, prediction_name), bbox_inches='tight', dpi=800)
    if show:
        plt.show()
    plt.clf()
    plt.cla()
    plt.close()

### For calculating total pixel ratio

In [5]:
def total_pixel(config):
    label = sorted(glob.glob(config["dataset_dir"]+"label/*.ras"))
    uni_value = []
    total_pix = 0
    water = 0
    for i in range(len(label)):
        mask = readRas(label[i])[0]
        for i in np.unique(mask):
            if i not in uni_value:
                uni_value.append(i)
        mask[mask==255] = 1
        total_pix += mask.shape[0]*mask.shape[1]
        water += np.sum(mask)


    print("unique value", uni_value)
    pixels = {"Water":water/total_pix, 
            "NON-Water":(total_pix - water)/total_pix}
    print(pixels)
    
    
total_pixel(config)

unique value [0, 255]
{'Water': 0.23953188267219586, 'NON-Water': 0.7604681173278042}


### Pixel percentage check

In [20]:
with open(config['p_train_dir'], 'r') as j:
    train_dir = json.loads(j.read())

train_idx = train_dir['patch_idx']

# Augment
# augment_obj = Augment(config['batch_size'], config['in_channels'], tiles_width=config['tiles_size'])
# n_batch_size = config['batch_size']-augment_obj.aug_img_batch

# # weights
# weights = tf.constant(config['balance_weights'])
# n_batch_size = config['batch_size'] - augment_obj.aug_img_batch


# train_dataset = MyDataset(train_dir,
#                         in_channels=config['in_channels'], patchify=config['patchify'],
#                         batch_size=n_batch_size, transform_fn=transform_data, 
#                         num_class=config['num_classes'], augment=augment_obj, 
#                         weights=weights, patch_idx=train_idx, tile_width=config["tiles_size"])


train_dataset = MyDataset(train_dir,
                        in_channels=config['in_channels'], patchify=True,
                        batch_size=10, transform_fn=transform_data, 
                        num_class=config['num_classes'], augment=False, 
                        weights=None, patch_idx=train_idx, tile_width=config["tiles_size"])

In [21]:
train_dataset.__len__()

0

In [13]:
for x, y in train_dataset:
    print(x.shape)
    print(y.shape)
    break


### Plotting Row Data

In [6]:
def display_raw_data(config):
    
    pathlib.Path((config['visualization_dir']+'display_raw_data')).mkdir(parents = True, exist_ok = True)

    wid, hgt = config['actual_width'], config['actual_height']  # from the masterpar.xml

    rslcs = sorted(glob.glob(config["dataset_dir"]+"rslc/*.rslc.notopo"))
    label = sorted(glob.glob(config["dataset_dir"]+"label/*.ras"))

    rslc1 = data_reader.readBin(rslcs[0], width=wid, dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(rslcs[1], width=wid, dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(label[0])[0]
    rslc2_label = readRas(label[1])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1

    id = rslcs[0].split("/")[-1].split(".")[0].split("\\")[-1]
    id2 = rslcs[3].split("/")[-1].split(".")[0].split("\\")[-1]

    display_list = {
                     "rslc1": np.abs(rslc1),
                     "rslc2": np.abs(rslc2),
                     "ifg": np.angle(rslc2*np.conjugate(rslc1)),
                     "rslc1_label": rslc1_label,
                     "rslc2_label": rslc2_label
                     }
    plot(display_list, (id+"_"+id2), (config['visualization_dir']+'display_raw_data'))

display_raw_data(config)

### Plotting Tiles

Plot a single tiles from train, test and validation 

In [7]:
def display_all(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """
    pathlib.Path((config['visualization_dir']+'display_tiles')).mkdir(parents = True, exist_ok = True)

    rslc1 = data_reader.readBin(data['rslc0'], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(data['rslc1'], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(data['rslc0_label'])[0]
    rslc2_label = readRas(data['rslc1_label'])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1
        
    id = data['rslc1'].split("/")[-1].split(".")[0].split("\\")[-1]
    # ifg = np.angle(np.log(np.abs(rslc2))*np.conjugate(np.log(np.abs(rslc1))))
    ifg = np.angle(rslc2*np.conjugate(rslc1))

    print('rslc1:',rslc1.shape)
    print('rslc2:',rslc2.shape)
    print('ifg:',ifg.shape)
    print('rslc1_label:',rslc1_label.shape)
    print('rslc2_label:',rslc2_label.shape)

    display_list = {
                   "rslc1":np.abs(rslc1),
                   "rslc2":np.abs(rslc2),
                   "ifg":ifg,
                   "rslc1_label": rslc1_label,
                   "rslc2_label":rslc2_label
                  }
    plot(display_list, id, (config['visualization_dir']+'display_tiles'))


# change the value for specific plot
train_index = 5
val_index = 5
test_index = 1

train_dir = pd.read_csv(config['train_dir'])
print("Train examples: ", len(train_dir))
display_all(train_dir.iloc[train_index], config)
    
valid_dir = pd.read_csv(config['valid_dir'])
print("Valid examples: ", len(valid_dir))
display_all(valid_dir.iloc[val_index], config)

# test_dir = pd.read_csv(config['test_dir'])
# print("Test examples: ", len(test_dir))
# display_all(test_dir.iloc[test_index], config)

Train examples:  104
rslc1: (1024, 1024)
rslc2: (1024, 1024)
ifg: (1024, 1024)
rslc1_label: (1024, 1024)
rslc2_label: (1024, 1024)


Valid examples:  13
rslc1: (1024, 1024)
rslc2: (1024, 1024)
ifg: (1024, 1024)
rslc1_label: (1024, 1024)
rslc2_label: (1024, 1024)


#### Plot all the tiles

In [8]:
def display_all(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """
    pathlib.Path((config['visualization_dir']+'display_tiles')).mkdir(parents = True, exist_ok = True)
    
    pbar = ProgressBar()
    for i in pbar(range(len(data))):
        rslc1 = data_reader.readBin(data['rslc0'][i], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
        rslc2 = data_reader.readBin(data['rslc1'][i], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
        rslc1_label = readRas(data['rslc0_label'][i])[0]
        rslc2_label = readRas(data['rslc1_label'][i])[0]

        rslc1_label[rslc1_label==255] = 1
        rslc2_label[rslc2_label==255] = 1
        
        id = data['rslc1'][i].split("/")[-1].split(".")[0].split("\\")[-1]
        # ifg = np.angle(np.log(np.abs(rslc2))*np.conjugate(np.log(np.abs(rslc1))))
        ifg = np.angle(rslc2*np.conjugate(rslc1))

        display_list = {
                        "rslc1":np.abs(rslc1),
                        "rslc2":np.abs(rslc2),
                        "ifg":ifg,
                        "rslc1_label": rslc1_label,
                        "rslc2_label":rslc2_label
                        }
        plot(display_list, id, (config['visualization_dir']+'display_tiles'))



# train_dir = pd.read_csv(config['train_dir'])
# display_all(train_dir, config)
    
valid_dir = pd.read_csv(config['valid_dir'])
display_all(valid_dir, config)

test_dir = pd.read_csv(config['test_dir'])
display_all(test_dir, config)

100% |########################################################################|
100% |########################################################################|


### Plot Patch image

#### Plot a single patch image

In [ ]:
def display_patch(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """

    pathlib.Path((config['visualization_dir']+'display_patch')).mkdir(parents = True, exist_ok = True)

    rslc1 = data_reader.readBin(data['rslc0'], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(data['rslc1'], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(data['rslc0_label'])[0]
    rslc2_label = readRas(data['rslc1_label'])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1
        
    rslc1_amp = rslc1[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_amp = rslc2[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc1_label_amp = rslc1_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_label_amp = rslc2_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
        
    id = data['rslc1'].split("/")[-1].split(".")[0].split("\\")[-1]
    ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))

    print('rslc1:',rslc1_amp.shape)
    print('rslc2:',rslc2_amp.shape)
    print('ifg:',ifg.shape)
    print('rslc1_label:',rslc1_label_amp.shape)
    print('rslc2_label:',rslc2_label_amp.shape)
    
    
    display_list = {
                    "rslc1":np.abs(rslc1_amp),
                    "rslc2":np.abs(rslc2_amp),
                    "ifg":ifg,
                    "rslc1_label": rslc1_label_amp,
                    "rslc2_label":rslc2_label_amp
                    }
    plot(display_list, id, (config['visualization_dir']+'display_patch'))
            
# change the value for specific plot
train_index = 1
val_index = 1
test_index = 1


with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/train_patch_phr_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df.iloc[train_index], config)
    
with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/valid_patch_phr_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df.iloc[val_index], config)
    
# with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
#         patch_dir = json.loads(j.read())
# test_patch_df = pd.DataFrame.from_dict(patch_dir)
# print("Test examples: ",len(test_patch_df))
# display_patch(test_patch_df.iloc[test_index], config)

#### Plot all the patch images

In [ ]:
def display_patch(data, config):
        """
        Summary:
            save all images into single figure
        Arguments:
            data : data file holding images path
            directory (str) : path to save images
        Return:
            save images figure into directory
        """
        
        pathlib.Path((config['visualization_dir']+'display_patch')).mkdir(parents = True, exist_ok = True)
        pbar = ProgressBar()
        for i in pbar(range(len(data))):
            rslc1 = data_reader.readBin(data['rslc0'][i], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
            rslc2 = data_reader.readBin(data['rslc1'][i], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
            rslc1_label = readRas(data['rslc0_label'][i])[0]
            rslc2_label = readRas(data['rslc1_label'][i])[0]
    
            rslc1_label[rslc1_label==255] = 1
            rslc2_label[rslc2_label==255] = 1
            
            rslc1_amp = rslc1[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_amp = rslc2[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc1_label_amp = rslc1_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_label_amp = rslc2_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            
            id = data['rslc1'][i].split("/")[-1].split(".")[0].split("\\")[-1]
            ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
            
            display_list = {
                            "rslc1":np.abs(rslc1_amp),
                            "rslc2":np.abs(rslc2_amp),
                            "ifg":ifg,
                            "rslc1_label": rslc1_label_amp,
                            "rslc2_label":rslc2_label_amp
                            }
            plot(display_list, id, (config['visualization_dir']+'display_patch'), show=False)
            
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/train_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/valid_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
test_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Test examples: ",len(test_patch_df))
display_patch(test_patch_df, config)

#### Plot a single patch balance image

In [17]:
def display_patch(data, config):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """

    pathlib.Path((config['visualization_dir']+'display_patch_balance')).mkdir(parents = True, exist_ok = True)

    rslc1 = data_reader.readBin(data['rslc0'], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
    rslc2 = data_reader.readBin(data['rslc1'], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
    rslc1_label = readRas(data['rslc0_label'])[0]
    rslc2_label = readRas(data['rslc1_label'])[0]

    rslc1_label[rslc1_label==255] = 1
    rslc2_label[rslc2_label==255] = 1
        
    rslc1_amp = rslc1[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_amp = rslc2[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc1_label_amp = rslc1_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
    rslc2_label_amp = rslc2_label[data['patch_idx'][0]:data['patch_idx'][1], data['patch_idx'][2]:data['patch_idx'][3]]
        
    id = data['rslc1'].split("/")[-1].split(".")[0].split("\\")[-1]
    # ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
    ifg = np.angle(rslc1_amp*np.conjugate(rslc2_amp))
        
    display_list = {
                    "rslc1":np.abs(rslc1_amp),
                    "rslc2":np.abs(rslc2_amp),
                    "ifg":ifg,
                    "rslc1_label": rslc1_label_amp,
                    "rslc2_label":rslc2_label_amp
                    }
    plot(display_list, id, (config['visualization_dir']+'display_patch_balance'))
            
# change the value for specific plot
train_index = 10
val_index = 10
test_index = 1


with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/train_patch_phr_cb_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df.iloc[train_index], config)
    
with open("/mnt/hdd2/mdsamiul/waterbody_detection_complex_data/data/json/valid_patch_phr_cb_512.json", 'r') as j:
        patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df.iloc[val_index], config)
    
# with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
#         patch_dir = json.loads(j.read())
# test_patch_df = pd.DataFrame.from_dict(patch_dir)
# print("Test examples: ",len(test_patch_df))
# display_patch(test_patch_df.iloc[test_index], config)

Train examples:  2856
Valid examples:  319


#### Plot all the patch balance images

In [ ]:
def display_patch(data, config):
        """
        Summary:
            save all images into single figure
        Arguments:
            data : data file holding images path
            directory (str) : path to save images
        Return:
            save images figure into directory
        """
        
        pathlib.Path((config['visualization_dir']+'display_patch_balance')).mkdir(parents = True, exist_ok = True)
        pbar = ProgressBar()
        for i in pbar(range(len(data))):
            rslc1 = data_reader.readBin(data['rslc0'][i], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
            rslc2 = data_reader.readBin(data['rslc1'][i], width=config['tiles_size'], dataType='floatComplex') # SLC(RSLC) image
            rslc1_label = readRas(data['rslc0_label'][i])[0]
            rslc2_label = readRas(data['rslc1_label'][i])[0]
    
            rslc1_label[rslc1_label==255] = 1
            rslc2_label[rslc2_label==255] = 1
            
            rslc1_amp = rslc1[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_amp = rslc2[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc1_label_amp = rslc1_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            rslc2_label_amp = rslc2_label[data['patch_idx'][i][0]:data['patch_idx'][i][1], data['patch_idx'][i][2]:data['patch_idx'][i][3]]
            
            id = data['rslc1'][i].split("/")[-1].split(".")[0].split("\\")[-1]
            ifg = np.angle(np.log(np.abs(rslc2_amp))*np.conjugate(np.log(np.abs(rslc1_amp))))
            
            display_list = {
                            "rslc1":np.abs(rslc1_amp),
                            "rslc2":np.abs(rslc2_amp),
                            "ifg":ifg,
                            "rslc1_label": rslc1_label_amp,
                            "rslc2_label":rslc2_label_amp
                            }
            plot(display_list, id, (config['visualization_dir']+'display_patch_balance'), show=False)
            
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/train_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
train_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Train examples: ", len(train_patch_df))
display_patch(train_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/valid_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
valid_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Valid examples: ", len(valid_patch_df))
display_patch(valid_patch_df, config)
    
with open("/home/mdsamiul/github_project/waterbody_segmentation_complex_data/data/json/test_patch_phr_512.json", 'r') as j:
    patch_dir = json.loads(j.read())
test_patch_df = pd.DataFrame.from_dict(patch_dir)
print("Test examples: ",len(test_patch_df))
display_patch(test_patch_df, config)